In [ ]:
import pandas as pd
# import matplotlib as plt
import numpy as np
# import seaborn as sns
# from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model # type: ignore
from tensorflow.keras.layers import LSTM, Dense # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore
# from tensorflow.keras.callbacks import ModelCheckpoint # type: ignore
# from keras import metrics

In [ ]:
df = pd.read_csv('Data_In.csv')
df.rename(columns = {'company_name' : 'Company'}, inplace = True)

# Print Rows with Null Values
# df[df.isnull().any(axis=1)]

In [ ]:
factor = 'Company'
search_term = 'AMAZON'

# Filter rows where 'Type' is equal to 'Health'
filtered_df = df[df[factor] == search_term]
filtered_df.shape


In [ ]:
# Reshape data into a 1D array
# Scale data to the min and max of the rows i.e 0 -1 range
# Flatten the data once again to a 1D array
# ====================================================================

data = filtered_df['Close'].values.reshape(-1, 1)
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)
# data_scaled[:10]

In [ ]:
def create_sequences(data, seq_length):
    sequences, labels = [], []
    
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length, 0]
        label = data[i:i + seq_length, 0]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

In [ ]:
seq_length = 1
n_samples = len(data_scaled)
sequences, labels = create_sequences(data_scaled, seq_length)
split = int(0.9 * len(sequences))
X_test, y_test = sequences[split:], labels[split:]
X_train, y_train = sequences[:split], labels[:split]

In [ ]:
model_lstm = Sequential()
model_lstm.add(LSTM(100, activation='relu', input_shape=(seq_length, 1), return_sequences=True))
model_lstm.add(LSTM(50, activation='relu'))
# model_lstm.add(Dense(320))
model_lstm.add(Dense(200))
model_lstm.add(Dense(1))
model_lstm.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

In [ ]:
model_lstm.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
model_lstm.save('model_lstm_7.keras')

In [103]:
predictor = load_model('model_lstm_7.keras')
amazn = predictor.predict(X_test)

21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step


In [104]:
amazn = scaler.inverse_transform(amazn)
amazn_y = scaler.inverse_transform(y_test.reshape(-1, 1))

In [106]:
dates_test = filtered_df['Date'].iloc[split + seq_length:]
df_plot_lstm = pd.DataFrame({'Date': dates_test[:len(amazn)], 'Real': amazn_y.flatten(), 'Predict': amazn.flatten()})

df_plot_lstm.tail(10)
# df_plot_lstm.to_csv('AMAZON.csv')

,Date,Real,Predict
31946,2023-12-15,147.419998,147.597336
31947,2023-12-18,149.970001,150.138992
31948,2023-12-19,154.070007,154.237717
31949,2023-12-20,153.789993,153.957306
31950,2023-12-21,152.119995,152.286469
31951,2023-12-22,153.839996,154.007385
31952,2023-12-26,153.419998,153.586914
31953,2023-12-27,153.410004,153.576904
31954,2023-12-28,153.339996,153.506836
31955,2023-12-29,153.380005,153.546875
